This quality control (QC) step is mainly focused on identifying and filtering out low quality cells and genes. Please refer to [Chapter 6: Quality Control](https://www.sc-best-practices.org/preprocessing_visualization/quality_control.html) for more information about QC in single-cell RNA-seq data.

> Heumos, L., Schaar, A.C., Lance, C. et al. Best practices for single-cell analysis across modalities. Nat Rev Genet (2023). [https://doi.org/10.1038/s41576-023-00586-w](https://doi.org/10.1038/s41576-023-00586-w)

## Import libraries

In [ ]:
import scanpy as sc
import anndata as ad
import seaborn as sns
import numpy as np
from scipy.stats import median_abs_deviation
import pooch
import toml

with open("../scanpy_config.toml", "r") as file:
    config = toml.load(file)

sc.settings.set_figure_params(dpi=300, facecolor="white")
sc.settings.n_jobs = 4


def is_outlier(adata, metric: str, nmads: int):
    M = adata.obs[metric]
    lower_bound = np.median(M) - nmads * median_abs_deviation(M)
    upper_bound = np.median(M) + nmads * median_abs_deviation(M)
    outlier = (M < lower_bound) | (upper_bound < M)
    return outlier

## Loading data

In [ ]:
#| warning: false
EXAMPLE_DATA = pooch.create(
    path=pooch.os_cache("scverse_tutorials"),
    base_url="doi:10.6084/m9.figshare.22716739.v1/",
)
EXAMPLE_DATA.load_registry_from_doi()

samples = {
    "s1d1": "s1d1_filtered_feature_bc_matrix.h5",
    "s1d3": "s1d3_filtered_feature_bc_matrix.h5",
}
adatas = {}

for sample_id, filename in samples.items():
    path = EXAMPLE_DATA.fetch(filename)
    sample_adata = sc.read_10x_h5(path)
    sample_adata.var_names_make_unique()
    adatas[sample_id] = sample_adata

adata = ad.concat(adatas, label="sample")
adata.obs_names_make_unique()

## Calculating quality control metrics

In [ ]:
# mitochondrial genes, "MT-" for human, "Mt-" for mouse
if config['basic']['organism']=="human":
    mt_prefix = "MT-"
elif config['basic']['organism']=="mouse":
    mt_prefix =  "Mt-"
adata.var["mt"] = adata.var_names.str.startswith(mt_prefix)
# ribosomal genes
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
# hemoglobin genes
adata.var["hb"] = adata.var_names.str.contains("^HB[^(P)]")

sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt", "ribo", "hb"], inplace=True, percent_top=[20], log1p=True
)

# Filtering low quality cells

## Visualizing quality control metrics

In [ ]:
#| layout-nrow: 2
p1 = sns.displot(adata.obs["total_counts"],
                 bins=100, kde=False)
p2 = sc.pl.scatter(adata,
                  "total_counts",
                  "n_genes_by_counts",
                  color="pct_counts_mt")
p3 = sc.pl.violin(adata,
                  "pct_counts_mt",
                  jitter=0.4)
p4 = sc.pl.violin(adata,
                  "n_genes_by_counts",
                  jitter=0.4,)

## Identifying cells with low quality

In [ ]:
nmad_log1p_total_counts = 5
nmad_log1p_n_genes_by_counts = 5
nmad_pct_counts_in_top_20_genes = 5

adata.obs["outlier"] = (
    is_outlier(
        adata, "log1p_total_counts",
        nmad_log1p_total_counts)
    | is_outlier(
        adata, "log1p_n_genes_by_counts",
        nmad_log1p_n_genes_by_counts)
    | is_outlier(
        adata, "pct_counts_in_top_20_genes",
        nmad_pct_counts_in_top_20_genes)
)
n_outliers = adata.obs.outlier.value_counts()[True]
pct_outliers = n_outliers / adata.n_obs * 100
print(f"Number of outliers:     {n_outliers} / {adata.n_obs}")
print(f"Percentage of outliers: {pct_outliers:.2f}%")

## Identifying mitochondrial outliers

In [ ]:
nmad_pct_counts_mt = 3
threshold_pct_counts_mt = 8
adata.obs["mt_outlier"] = is_outlier(
    adata, "pct_counts_mt", nmad_pct_counts_mt) | (
    adata.obs["pct_counts_mt"] > threshold_pct_counts_mt
)
n_outliers = adata.obs.mt_outlier.value_counts()[True]
pct_outliers = n_outliers / adata.n_obs * 100
print(f"Number of outliers:     {n_outliers} / {adata.n_obs}")
print(f"Percentage of outliers: {pct_outliers:.2f}%")

## Filtering

In [ ]:
print(f"Number of cells before filtering: {adata.n_obs}")
adata = adata[(~adata.obs.outlier) & (~adata.obs.mt_outlier)].copy()
print(f"Number of cells after filtering:  {adata.n_obs}")

## Quality control metrics after filtering

In [ ]:
#| layout-nrow: 2
p1 = sns.displot(adata.obs["total_counts"],
                 bins=100, kde=False)
p2 = sc.pl.scatter(adata,
                  "total_counts",
                  "n_genes_by_counts",
                  color="pct_counts_mt")
p3 = sc.pl.violin(adata,
                  "pct_counts_mt",
                  jitter=0.4)
p4 = sc.pl.violin(adata,
                  "n_genes_by_counts",
                  jitter=0.4,)

# Correction of ambient RNA


In [ ]:
import anndata2ri
import logging

import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

In [ ]:
adata_pp = adata.copy()
sc.pp.normalize_per_cell(adata_pp)
sc.pp.log1p(adata_pp)
sc.pp.pca(adata_pp)
sc.pp.neighbors(adata_pp)
sc.tl.leiden(adata_pp, key_added="soupx_groups")

# Preprocess variables for SoupX
soupx_groups = adata_pp.obs["soupx_groups"]
del adata_pp
cells = adata.obs_names
genes = adata.var_names
data = adata.X.T

adata_raw = ad.concat(adatas, label="sample")
adata_raw.var_names_make_unique()
data_tod = adata_raw.X.T
del adata_raw

```{r}
library(SoupX)
```